In [1]:
import warnings
warnings.filterwarnings("ignore")
import requests
import folium
from c import json_normalize

In [2]:
# 전국 또는 특정 지역의 스타벅스 매장 위치를 찾아와서 지도위에 표시하기
# 1. requests로 스타벅스 매장 위치 데이터를 가져와서 딕셔너리 타입으로 변환한다.
# 2. json_normalize() 메소드로 json 타입이 변환된 딕셔너리를 판다스 데이터프레임으로 변환한다.
# 3. folium 모듈을 사용해서 지도를 표시하고 지도위의 스타벅스 매장 위치에 마커를 표시한다.

In [7]:
# 서울 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=KO1XF3AWAI'
request = requests.post(targetSite, data={
    'ins_lat'   : 37.5108295,
    'ins_lng'   : 127.02928809999999,
    'p_sido_cd' : '01',
    'p_gugun_cd': '',
    'in_biz_cd' : '',
    'iend'      : 1600,
    'set_date'  : ''
})
storeList = request.json()
print(len(storeList['list']))
star_json = request.json()
print(type(star_json))

545
<class 'dict'>


In [8]:
# json_normalize 모듈의 json_normalize() 함수로 json 타입이 변환된 딕셔너리를 판다스 데이터프레임으로 변환한다.
# json_normalize(데이터프레임으로 변환할 데이터가 저장된 딕셔너리, 딕셔너리 중에서 데이터프레임으로 변환할 키)
star_df = json_normalize(star_json, 'list')
print(type(star_df))

<class 'pandas.core.frame.DataFrame'>


In [9]:
star_df

,seq,sido_cd,sido_nm,gugun_cd,gugun_nm,code_order,view_yn,store_num,sido,gugun,...,p40,p80,t22,t21,p90,t05,t30,t36,t27,t43
0,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
1,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
2,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
3,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
4,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
541,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
542,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
543,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# 작업에 필요한 컬럼 몇 가지를 선택해서 지도에 마커를 표시할 때 사용할 데이터가 저장된 데이터프레임을 ㅁ나든다.
# s_name       => 지점 이름
# sido_code    => 시도 코드
# sido_name    => 시도 이름
# gugun_code   => 구군 코두
# gugun_name   => 구군 이름
# doro_address => 도로명 주소
# lat          => 위도
# lot          => 경도

# 판다스 데이터프레임에서 특정 컬럼의 데이터를 얻어올 때 얻어올 컬럼의 개수가 1개라면 컬럼 이름만 적으면 되지만 얻어올 컬럼이
# 2개 이상일 경우 얻어올 컬럼 이름을 리스트로 만들어서 얻어와야 한다.
star_df_map = star_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address', 'lat', 'lot']]
star_df_map

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
0,역삼아레나빌딩,01,서울,0101,강남구,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069
1,논현역사거리,01,서울,0101,강남구,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223
2,신사역성일빌딩,01,서울,0101,강남구,서울특별시 강남구 강남대로 584 (논현동),37.514132,127.020563
3,국기원사거리,01,서울,0101,강남구,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495
4,스탈릿대치R,01,서울,0101,강남구,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583
...,...,...,...,...,...,...,...,...
540,사가정역,01,서울,0125,중랑구,서울특별시 중랑구 면목로 310,37.579594,127.087966
541,상봉역,01,서울,0125,중랑구,"서울특별시 중랑구 망우로 307, ,3,4번지 (상봉동)",37.59689,127.08647
542,묵동이마트,01,서울,0125,중랑구,"서울특별시 중랑구 동일로 932 (묵동, 묵동자이아파트) (묵동이마트 B1층)",37.613433,127.077484
543,묵동,01,서울,0125,중랑구,서울특별시 중랑구 동일로 952,37.615368,127.076633


In [12]:
star_df_map.dtypes

s_name          object
sido_code       object
sido_name       object
gugun_code      object
gugun_name      object
doro_address    object
lat             object
lot             object
dtype: object

In [13]:
# astype() 메소드로 위도, 경도의 데이터 타입을 object에서 float로 변환한다.
star_df_map['lat'] = star_df_map['lat'].astype(float)
star_df_map['lot'] = star_df_map['lot'].astype(float)
star_df_map.dtypes

s_name           object
sido_code        object
sido_name        object
gugun_code       object
gugun_name       object
doro_address     object
lat             float64
lot             float64
dtype: object

In [14]:
# 종각점
star_df_map[star_df_map['s_name'] == '종각']

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
402,종각,01,서울,0118,종로구,서울특별시 종로구 종로 64 (종로2가),37.569918,126.984528


In [15]:
# 종로관철
star_df_map[star_df_map['s_name'] == '종로관철']

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
404,종로관철,01,서울,0118,종로구,"서울특별시 종로구 종로12길 21, 2층 (관철동)",37.569058,126.986013
